# Analysis for DataCrumbs

This is a simple analysis notebook for Datacrumbs.

## Imports

In [1]:
import logging
import json
import dask
import os
from pathlib import Path
from glob import glob
import math
import zindex_py as zindex
import numpy as np
import intervals as I
import pandas as pd
from tqdm.notebook import trange, tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster, progress, wait, get_client
from dask.distributed import Future, get_client

## Project Variables

In [3]:
app_root = str(Path(os.getcwd()).parent)

In [4]:
logging.basicConfig(
    level=logging.INFO,
    handlers=[
        logging.StreamHandler(),
    ],
    format="%(asctime)s [%(levelname)s]: %(message)s in %(pathname)s:%(lineno)d",
)

## Setup Dask Local Cluster

In [5]:
workers=16
cluster = LocalCluster(n_workers=workers)  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
logging.info(f"Initialized Client with {workers} workers and link {client.dashboard_link}")

2024-11-10 12:37:37,089 [INFO]: Initialized Client with 16 workers and link http://127.0.0.1:8787/status in /var/tmp/haridev/ipykernel_359281/3142773904.py:4


## Start Analysis

In [6]:

import os 
is_trace = True
folder="/usr/workspace/haridev/xio/"
file=f"{folder}/logs/ior/*.pfw.gz"
file_pattern = glob(file)
file_pattern

['/usr/workspace/haridev/xio//logs/ior/ior_4k.pfw.gz',
 '/usr/workspace/haridev/xio//logs/ior/ior_16k-64k.pfw.gz',
 '/usr/workspace/haridev/xio//logs/ior/ior-1mb-4mb.pfw.gz',
 '/usr/workspace/haridev/xio//logs/ior/ior-256k.pfw.gz',
 '/usr/workspace/haridev/xio//logs/ior/ior-256k-multiple-interfaces.pfw.gz',
 '/usr/workspace/haridev/xio//logs/ior/ior-hdf5-256k.pfw.gz']

## Function to load trace data

In [7]:
def create_index(filename):
    index_file = f"{filename}.zindex"
    if not os.path.exists(index_file):
        status = zindex.create_index(filename, index_file=f"file:{index_file}",
                                     regex="id:\b([0-9]+)", numeric=True, unique=True, debug=False, verbose=False)
        logging.debug(f"Creating Index for {filename} returned {status}")
    return filename

def get_linenumber(filename):
    index_file = f"{filename}.zindex"
    line_number = zindex.get_max_line(filename, index_file=index_file, debug=False, verbose=False)
    logging.debug(f" The {filename} has {line_number} lines")
    return (filename, line_number)

def get_size(filename):
    if filename.endswith('.pfw'):
        size = os.stat(filename).st_size
    elif filename.endswith('.pfw.gz'):
        index_file = f"{filename}.zindex"
        line_number = zindex.get_max_line(filename, index_file=index_file,debug=False, verbose=False)
        size = line_number * 256
    logging.debug(f" The {filename} has {size/1024**3} GB size")
    return int(size)


def generate_line_batches(filename, max_line):
    batch_size = 16*1024
    for start in range(0, max_line, batch_size):
        end =  min((start + batch_size - 1) , (max_line - 1))
        logging.debug(f"Created a batch for {filename} from [{start}, {end}] lines")
        yield filename, start, end

def load_indexed_gzip_files(filename, start, end):
    index_file = f"{filename}.zindex"
    json_lines = zindex.zquery(filename, index_file=index_file,
                          raw=f"select a.line from LineOffsets a where a.line >= {start} AND a.line <= {end};", debug=False, verbose=False)
    logging.debug(f"Read {len(json_lines)} json lines for [{start}, {end}]")
    return json_lines

In [8]:
def load_profile(line):
    d = {}
    if line is not None and line !="" and len(line) > 0 and "[" != line[0] and line != "\n" :
        try:
            unicode_line = ''.join([i if ord(i) < 128 else '#' for i in line])
            val = json.loads(unicode_line)
            if "pid" in d:
                d["pid"] = val["pid"]
            if "tid" in d:
                d["tid"] = val["tid"]
            if "ts" in d:
                d["ts_us"] = int(val["ts"])
            d["filename"] = "NA"
            if "args" in val:
                if "time" in val["args"]:
                    d["dur_sec"] = float(val["args"]["time"])
                if "freq" in val["args"]:
                    d["freq"] = val["args"]["freq"]
                if "size_sum" in val["args"]:
                    d["size_bytes"] = val["args"]["size_sum"]
                if "fname" in val["args"] and val["args"]["fname"]:
                    d["filename"] = val["args"]["fname"]
            d["func_id"] = val["name"]
            d["cat"] = val["cat"]
        except Exception as error:
            logging.error(f"Processing {line} failed with {error}")
    return d


def load_trace(line):
    d = {}
    if line is not None and line !="" and len(line) > 0 and "[" != line[0] and line != "\n" :
        try:
            unicode_line = ''.join([i if ord(i) < 128 else '#' for i in line])
            val = json.loads(unicode_line)
            d["name"] = val["name"]
            d["cat"] = val["cat"]
            if "pid" in val:
                d["pid"] = val["pid"]
            if "tid" in val:
                d["tid"] = val["tid"]
            d["ts"] = 0
            d["dur"] = 0
            if "ts" in val:
                d["ts"] = int(val["ts"])
                d["te"] = int(val["ts"])
            d["dur"] = 1
            if "dur" in val:
                d["dur"] = int(val["dur"])
            if "args" in val and "hhash" in val["args"]:                    
                d["hhash"] = val["args"]["hhash"]
            if "ts" in val:
                interval = I.closedopen(d["ts"], d["ts"] + 1)
                if d["dur"] > 0:
                    d["te"] = int(val["ts"]) + d["dur"]
                    interval = I.closedopen(d["ts"], d["ts"] + d["dur"])
                d["interval"] = I.to_string(interval)
            if val["ph"] != "M":
                d["type"] = 0    
                if "args" in val:                    
                    if "hhash" in val["args"]:
                        d["hhash"] = val["args"]["hhash"]
                    if "size_sum" in val["args"]:
                        d["size"] = val["args"]["size_sum"]
                    if "fhash" in val["args"]:
                        d["fhash"] = val["args"]["fhash"]
            else:
                if val["name"] == "FH":
                    d["type"] = 1
                    if "args" in val:
                        if "name" in val["args"]:
                            d["name"] = val["args"]["name"]
                        if "value" in val["args"]:
                            d["hash"] = val["args"]["value"]
                elif val["name"] == "HH":
                    d["type"] = 2
                    if "args" in val:
                        if "name" in val["args"]:
                            d["name"] = val["args"]["name"]
                        if "value" in val["args"]:
                            d["hash"] = val["args"]["value"]
            
        except Exception as error:
            logging.error(f"Processing {line} failed with {error}")
    return d

## Create Dask Dataframe

In [9]:
if len(file_pattern) > 0:
    dask.bag.from_sequence(file_pattern).map(create_index).compute()
    logging.info(f"Created index for {len(file_pattern)} files")
    total_size = dask.bag.from_sequence(file_pattern).map(get_size).sum()
    n_partition = math.ceil(total_size.compute() / (128 * 1024 ** 2))
    logging.info(f"Total size of all files are {total_size} bytes")
    max_line_numbers = dask.bag.from_sequence(file_pattern).map(get_linenumber).compute()
    logging.info(f"Max lines per file are {max_line_numbers}")
    json_line_delayed = []
    total_lines = 0
    for filename, max_line in max_line_numbers:
        total_lines += max_line
        for _, start, end in generate_line_batches(filename, max_line):
            json_line_delayed.append((filename, start, end))

    logging.info(f"Loading {len(json_line_delayed)} batches out of {len(file_pattern)} files and has {total_lines} lines overall")
    json_line_bags = []
    for filename, start, end in json_line_delayed:
        num_lines = end - start + 1
        json_line_bags.append(dask.delayed(load_indexed_gzip_files, nout=num_lines)(filename, start, end))
    json_lines = dask.bag.concat(json_line_bags)
    if is_trace:
        pfw_bag = json_lines.map(load_trace).filter(lambda x: "name" in x)
    else:
        pfw_bag = json_lines.map(load_profile).filter(lambda x: "func_id" in x)
    pfw_bag.take(1)

2024-11-10 12:37:37,416 [INFO]: Created index for 6 files in /var/tmp/haridev/ipykernel_359281/864066620.py:3
2024-11-10 12:37:37,533 [INFO]: Total size of all files are <dask.bag.core.Item object at 0x15552128afd0> bytes in /var/tmp/haridev/ipykernel_359281/864066620.py:6
2024-11-10 12:37:37,588 [INFO]: Max lines per file are [('/usr/workspace/haridev/xio//logs/ior/ior_4k.pfw.gz', 7565244), ('/usr/workspace/haridev/xio//logs/ior/ior_16k-64k.pfw.gz', 9788674), ('/usr/workspace/haridev/xio//logs/ior/ior-1mb-4mb.pfw.gz', 34785028), ('/usr/workspace/haridev/xio//logs/ior/ior-256k.pfw.gz', 9397525), ('/usr/workspace/haridev/xio//logs/ior/ior-256k-multiple-interfaces.pfw.gz', 60400779), ('/usr/workspace/haridev/xio//logs/ior/ior-hdf5-256k.pfw.gz', 25153457)] in /var/tmp/haridev/ipykernel_359281/864066620.py:8
2024-11-10 12:37:37,615 [INFO]: Loading 8981 batches out of 6 files and has 147090707 lines overall in /var/tmp/haridev/ipykernel_359281/864066620.py:16


In [10]:
if is_trace:
    columns = {'hhash': "string[pyarrow]", 'pid': "uint64[pyarrow]", 'tid': "uint64[pyarrow]",
                'cat': "string[pyarrow]", 'name': "string[pyarrow]", 'type':  "uint8[pyarrow]",
            'ts': "uint64[pyarrow]", 'te': "uint64[pyarrow]", 'dur': "uint64[pyarrow]", 'interval': "string[pyarrow]", 
             'size': "uint64[pyarrow]", 'fhash': "string[pyarrow]", 'hash': "string[pyarrow]", 
           }
else:
    columns = {'pid': "uint64[pyarrow]", 'tid': "uint64[pyarrow]",
            'ts_us': "uint64[pyarrow]", 'dur_sec': "float32[pyarrow]", 
            'freq': "uint64[pyarrow]", 'size_bytes': "uint64[pyarrow]", 'name': "string[pyarrow]", 
            'filename': "string[pyarrow]", 
            'cat': "string[pyarrow]"}

In [11]:
events = pfw_bag.to_dataframe(meta=columns)

In [12]:
events = events.repartition(npartitions=n_partition).persist()
_ = wait(events)

In [13]:
fhash = events.query("type == 1")[["name","hash"]]
hhash = events.query("type == 2")[["name","hash"]]
event = events.query("type == 0")
fhashes = fhash.query("name.str.contains('test.bat')").compute()["hash"]
fhashes = fhashes.to_list()


## Analysis

In [14]:

interesting_events = event.query("fhash.isin(@value)", local_dict={"value": fhashes}).sort_values("ts")
interesting_events.compute()

,hhash,pid,tid,cat,name,type,ts,te,dur,interval,size,fhash,hash
3368,ecd9cccc050c9e893ab33b1a228fe76d,15882,15882,sys,openat,0,5448559896,5448937723,377827,"[5448559896,5448937723)",<NA>,3c30b6298abfeb121cd313effdb92256,<NA>
3374,ecd9cccc050c9e893ab33b1a228fe76d,15882,15882,sys,lseek,0,5448956059,5448965554,9495,"[5448956059,5448965554)",<NA>,3c30b6298abfeb121cd313effdb92256,<NA>
3609,ecd9cccc050c9e893ab33b1a228fe76d,15882,15882,sys,write,0,5448970686,5449450628,479942,"[5448970686,5449450628)",16384,3c30b6298abfeb121cd313effdb92256,<NA>
3613,ecd9cccc050c9e893ab33b1a228fe76d,15882,15882,sys,lseek,0,5449469108,5449473314,4206,"[5449469108,5449473314)",<NA>,3c30b6298abfeb121cd313effdb92256,<NA>
3858,ecd9cccc050c9e893ab33b1a228fe76d,15882,15882,sys,write,0,5449477719,5449874977,397258,"[5449477719,5449874977)",16384,3c30b6298abfeb121cd313effdb92256,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,ecd9cccc050c9e893ab33b1a228fe76d,252574,252574,sys,pread64,0,7715452830316,7715453991341,1161025,"[7715452830316,7715453991341)",262144,1.438620452325829e+19,<NA>
2302,ecd9cccc050c9e893ab33b1a228fe76d,252574,252574,sys,pread64,0,7715729832783,7715736439761,6606978,"[7715729832783,7715736439761)",262144,1.438620452325829e+19,<NA>
4070,ecd9cccc050c9e893ab33b1a228fe76d,252574,252574,sys,pread64,0,7715952253337,7715955873221,3619884,"[7715952253337,7715955873221)",262144,1.438620452325829e+19,<NA>
4437,ecd9cccc050c9e893ab33b1a228fe76d,252574,252574,sys,pread64,0,7715998108265,7716001726245,3617980,"[7715998108265,7716001726245)",262144,1.438620452325829e+19,<NA>


In [15]:
interesting_events["combined_name"] = interesting_events["name"] + "-" + interesting_events["cat"]
ts_events = interesting_events[["size"]].compute().reset_index().drop("index", axis=1)
ts_events["size"].fillna(value=1.0)

/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/dask_expr/_shuffle.py:1394: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mins = mins.bfill()
/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/dask_expr/_shuffle.py:1395: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  maxes = maxes.bfill()


2024-11-10 12:48:52,787 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 37391b081af65c2e45cbae88ab710187 initialized by task ('shuffle-transfer-37391b081af65c2e45cbae88ab710187', 131) executed on worker tcp://127.0.0.1:37081
2024-11-10 12:48:56,682 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 37391b081af65c2e45cbae88ab710187 deactivated due to stimulus 'task-finished-1731271736.6801343'
2024-11-10 12:49:01,471 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle dc3f970979eefcc3a5816445870e8dec initialized by task ('shuffle-transfer-dc3f970979eefcc3a5816445870e8dec', 115) executed on worker tcp://127.0.0.1:37481
2024-11-10 12:49:04,341 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle dc3f970979eefcc3a5816445870e8dec deactivated due to stimulus 'task-finished-1731271744.3392837'
2024-11-10 12:49:09,948 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle dbe2eb22ff8fe1d84f492ee06d99a2f2 initialized by task ('shuffle-transfer-dbe2eb22ff

0              1
1              1
2          16384
3              1
4          16384
           ...  
853147    262144
853148    262144
853149    262144
853150    262144
853151    262144
Name: size, Length: 853152, dtype: uint64[pyarrow]

In [16]:
interesting_intervals  = interesting_events[["interval","name"]].compute()
interesting_intervals

/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/dask_expr/_shuffle.py:1394: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mins = mins.bfill()
/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/dask_expr/_shuffle.py:1395: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  maxes = maxes.bfill()


,interval,name
3368,"[5448559896,5448937723)",openat
3374,"[5448956059,5448965554)",lseek
3609,"[5448970686,5449450628)",write
3613,"[5449469108,5449473314)",lseek
3858,"[5449477719,5449874977)",write
...,...,...
60,"[7715452830316,7715453991341)",pread64
2302,"[7715729832783,7715736439761)",pread64
4070,"[7715952253337,7715955873221)",pread64
4437,"[7715998108265,7716001726245)",pread64


In [17]:

# interesting_events["interval"] = interesting_events.apply(lambda x: I.to_string(I.closed(x["ts"], x["ts"]+x["dur"])), axis=1)

In [18]:
def group_func(df):
    val = I.empty()
    for index, value in df.items():
        if str(value) != 'NA':
            pad_interval = I.from_string(str(value), int)
            val = val.union(pad_interval)
    logging.debug(f"Grouped Range into {val}")
    return I.to_string(val)
def union_portions():
    return dd.Aggregation(
        'union_portions',
        chunk=lambda s: s.apply(group_func),
        agg=lambda s: s.apply(group_func)
    )
relevant_intervals = interesting_events[["interval"]].reduction(chunk=lambda s: s.apply(group_func), aggregate=lambda s1: s1.apply(group_func))["interval"].compute()
relevant_intervals = I.from_string(relevant_intervals, int)
relevant_intervals_list = list(relevant_intervals)
relevant_intervals_list[:10], len(relevant_intervals_list)

/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/dask_expr/_shuffle.py:1394: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mins = mins.bfill()
/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/dask_expr/_shuffle.py:1395: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  maxes = maxes.bfill()


([[5448559896,5448937723),
  [5448956059,5448965554),
  [5448970686,5449450628),
  [5449469108,5449473314),
  [5449477719,5449874977),
  [5449884999,5449888787),
  [5449892961,5450252184),
  [5450261489,5450264918),
  [5450268966,5450631486),
  [5450640553,5450644029)],
 422948)

In [19]:
min_ts = relevant_intervals_list[0].lower
max_te = relevant_intervals_list[-1].upper
min_ts, max_te

(5448559896, 7716304259171)

In [20]:
filtered_events = event.query(f"ts >= {min_ts - 1e5} and te <= {max_te + 1e5} and dur > 0")
# filtered_events = event

In [21]:
# filtered_events["combined_name"] = filtered_events["name"] + "-" + filtered_events["cat"]
# event_batch_per_sys_call = set()
# rows = list(interesting_intervals.iterrows())
# count = 0
# ops_map = {}
# ops_counter = 0
# for index, row in tqdm(rows):
#     interval = I.from_string(row["interval"], int)
#     ops = row["name"]
#     a_overlaps_b = f"(ts >= {interval.lower} and ts <=  {interval.upper}) or (te >= {interval.lower} and te <=  {interval.upper})"
#     b_overlaps_a = f"({interval.lower} >= ts and {interval.lower} <=  te) or ({interval.upper} >= ts and {interval.upper} <=  te)"
#     batch = filtered_events.query(f"{a_overlaps_b} or {b_overlaps_a}")["combined_name"].unique().compute()
#     event_batch_per_sys_call.update(batch)
#     count += 1


In [28]:
unique_functions = filtered_events.groupby(["cat","name"]).agg({"dur":"count"}).compute()
unique_functions

2024-11-10 13:11:43,068 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('concat-filter-lambda-load_indexed_gzip_files-load_trace-to_dataframe-c79a9f1fc3c9b5f0e9f19de3114f3631', 4529))" coro=<Worker.execute() done, defined at /usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-11-10 13:11:43,098 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('concat-filter-lambda-load_indexed_gzip_files-load_trace-to_dataframe-c79a9f1fc3c9b5f0e9f19de3114f3631', 4528))" coro=<Worker.execute() done, defined at /usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-11-10 13:11:43,098 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('concat-filter-lambda-load_indexed_gzip_files-load

dur
cat    name                             
kernel ext4_get_inode_loc        1252881
       ext4_set_aops                 647
       ext4_create                   533
       ext4_da_get_block_prep    1239086
       __mnt_want_write_file      102686
...                                  ...
hdf5   H5C_flush_cache                 1
       H5C_apply_candidate_list        1
       H5ES_term_package               1
       H5FD_locate_signature          15
       H5S_mpio_space_type          2108

[1356 rows x 1 columns]

In [35]:
function_cat = {}
ignore_user_categories = ['hdf5', 'c', 'mpi', "sys"]
for cat, value in unique_functions.index:
    if cat not in ignore_user_categories:
        if cat not in function_cat:
            function_cat[cat]=set()
        function_cat[cat].add(value)
function_cat.keys()

dict_keys(['kernel', 'sys', 'os_cache', 'vfs'])

In [37]:
import json
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError
function_file = f"{app_root}/datacrumbs/configs/function.json"
with open(function_file, 'w') as convert_file: 
     convert_file.write(json.dumps(function_cat, default=set_default))

In [ ]:
batch_columns_list = dask.compute(event_batch_per_sys_call)
batch_columns_list

2024-11-10 11:59:09,448 [ERROR]: Task exception was never retrieved
future: <Task finished name='Task-10104337' coro=<Client._gather.<locals>.wait() done, defined at /usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/distributed/client.py:2391> exception=AllExit()> in /collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/asyncio/base_events.py:1753
Traceback (most recent call last):
  File "/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/distributed/client.py", line 2400, in wait
    raise AllExit()
distributed.client.AllExit
/usr/workspace/haridev/datacrumbs/venv/lib/python3.9/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 863.67 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
merged_df = None
count = 0
for batch in tqdm(event_batch_per_sys_call):
    if merged_df is not None:
        merged_df = merged_df.merge(batch, how='outer', on="combined_name",suffixes=('', f"_{count}"))
    else:
        merged_df = batch
    count += 1

In [ ]:
merged_df.columns

In [ ]:
dataset = merged_df.transpose()
df = dataset.reset_index().drop("index", axis=1)
df["op_name"] = "UNKNOWN"
for key, value in ops_map.items():
    df["op_name"] = df["op_name"].mask(df["op"].eq(value), key)
df["op_name"].unique()

In [ ]:
final_dataset = df.merge(ts_events, how='outer', left_index=True, right_index=True)
final_dataset["BW"] = final_dataset[f"size"] / (1024**2) / (final_dataset[f"{ops}-sys"] / 1e9)
final_dataset.drop([f"{ops}-sys", "size"], inplace=True, axis=1)
final_dataset.columns

In [40]:
output_file
final_dataset["op"] = ops

In [41]:
final_dataset.to_json(path_or_buf=f"{output_file}",orient='records', lines=True)


In [11]:
output_file="/usr/workspace/haridev/xio/output/jslines/write_ops-64_ts-64m-RAW-DIRECT.pfw.gz.jsonl"
ops="write"

In [12]:
from glob import glob
files = glob(f"{output_file}")
final_dataset_l = []
for file in files:
    final_dataset_l.append(dd.read_json(file))
final_dataset = dd.concat(final_dataset_l).compute().reset_index().drop("index", axis=1)
final_dataset["BW"] = final_dataset[f"transfer_size"] / (1024**2) / (final_dataset[f"{ops}-sys"]/1e9)
final_dataset["op"] = ops
final_dataset.to_json(path_or_buf=f"{output_file}",orient='records', lines=True)


## make the number of relevance features dynamic.

1. Add up the importance score to reach 95%.
2. Add Transfer size
3. Split features into layers and do this analysis per layer.
4. Correlation
   1. correlation matrix.
   2. PCA
   3. Lasso Regression (L1)
   4. Auto regression
5. SHAPLEY value (feature importance)
   1. Tree SHAP
6. How portable are the interfaces (do not overfit)

1. Tanzima for better models


## models
- sequential training: gradient boost
- 